In [23]:
import pandas as pd
from tkinter import filedialog
from os import path

In [41]:
#загрузка данных прямым импортом
#boxes = pd.read_csv("boxes.csv",sep = ";", decimal = ",", encoding="cp1251")
#items = pd.read_csv("items.csv",sep = ";", decimal = ",", encoding="cp1251")
#orders = pd.read_csv("orders.csv",sep = ";",encoding="cp1251")

In [24]:
#загрузка данных диалогом

boxes_filepath = filedialog.askopenfilename(initialdir = "/",title = "Select boxes",filetypes = [("csv files","*.csv")])
items_filepath = filedialog.askopenfilename(initialdir = "path.dirname(boxes_filepath)",title = "Select items",filetypes = [("csv files","*.csv")])
orders_filepath = filedialog.askopenfilename(initialdir = "path.dirname(items_filepath)",title = "Select orders",filetypes = [("csv files","*.csv")])

boxes = pd.read_csv(boxes_filepath,sep = ";", decimal = ",", encoding="cp1251")
items = pd.read_csv(items_filepath,sep = ";", decimal = ",", encoding="cp1251")
orders = pd.read_csv(orders_filepath,sep = ";",encoding="cp1251")


In [25]:
#вычисление объёма
boxes[u'Объём'] = boxes[u'Длина,мм']*boxes[u'Ширина,мм']*boxes[u'Высота,мм']/1000000
boxes.head()

,Производитель,Бренд,Тип картона,ГОСТ,"Длина,мм","Ширина,мм","Высота,мм",Артикул,Картон,Объём
0,Факел гофроупаковка,FEFCO 0201,Трехслойный,7,380,253,237,23167000.0,Т-23 Бурый,22.78518
1,Факел гофроупаковка,FEFCO 0202,Трехслойный,600,600,400,400,24898000.0,Т-23 Бурый,96.00000
2,Факел гофроупаковка,FEFCO 0203,Трехслойный,38,380,304,285,23169000.0,Т-23 Бурый,32.92320
3,Факел гофроупаковка,FEFCO 0204,Трехслойный,17,380,285,228,23197000.0,Т-23 Бурый,24.69240
4,Факел гофроупаковка,FEFCO 0205,Трехслойный,38,380,304,285,25426913.0,Т-24 Бурый,32.92320


In [26]:
items.head()

,Груз,"Вес, кг","Длина,мм","Ширина,мм","Высота,мм"
0,1070084359 000,17.90,770.0,550.0,210.0
1,45531223,8.55,710.0,580.0,280.0
2,Q3985A,5.20,660.0,300.0,310.0
3,FM1-A153-030000,3.25,640.0,485.0,180.0
4,8065B001AA 000,1.75,630.0,140.0,130.0


In [27]:
orders

,Заказ,Груз
0,k1,02351WTU
1,k2,02351XMT
2,k2,02351WDU
3,u1,Unknown
4,u2,02352HTF
5,u2,Unknown


In [16]:
orders = orders.merge(items, how = "left", on=u'Груз')
orders.set_index(u'Заказ', inplace = True)
orders.sort_index(inplace=True)
orders[u'Объём'] = orders[u'Длина,мм']*orders[u'Ширина,мм']*orders[u'Высота,мм']/1000000

In [17]:
#получаем список уникальных заказов для которых нашлись позиции
orders_list = set(orders[orders[u'Объём'].notnull()].index.difference(orders[orders[u'Объём'].isnull()].index))

#создаём пустой датафрейм и пустой список заказов-болванку
selected_boxes = pd.DataFrame(columns = boxes.columns)
ord_list = []

#начинаем цикл

for order_name in orders_list:

    #кладём название заказа в список
    ord_list.append(order_name)

    #определяем одна у нас позиция в заказе или нет
    dimention = orders.loc[order_name][u'Длина,мм'].ndim

    #ищем все коробки не меньших габаритов и объёма
    if dimention == 0:
        l = boxes.index[boxes[u'Длина,мм']>(orders.loc[order_name][u'Длина,мм'])]
        w = boxes.index[boxes[u'Ширина,мм']>(orders.loc[order_name][u'Ширина,мм'])]
        h = boxes.index[boxes[u'Высота,мм']>(orders.loc[order_name][u'Высота,мм'])]
        v = boxes.index[boxes[u'Объём']>(orders.loc[order_name][u'Объём'])]
        i = l.intersection(w).intersection(h).intersection(v)
    
    else:
        l = boxes.index[boxes[u'Длина,мм']>max(orders.loc[order_name][u'Длина,мм'])]
        w = boxes.index[boxes[u'Ширина,мм']>max(orders.loc[order_name][u'Ширина,мм'])]
        h = boxes.index[boxes[u'Высота,мм']>max(orders.loc[order_name][u'Высота,мм'])]
        v = boxes.index[boxes[u'Объём']>sum(orders.loc[order_name][u'Объём'])]
        i = l.intersection(w).intersection(h).intersection(v)    

    #вибираем коробку нименьшего объёма и добавляем её в список коробок
    selected_box = boxes.iloc[[boxes.iloc[i][u'Объём'].idxmin()]]

    selected_boxes = selected_boxes.append(selected_box)
        
#добавляем к датафрейму номер заказа и делаем по нему индекс
selected_boxes[u'Заказ'] = ord_list

selected_boxes.set_index(u'Заказ', inplace = True)

#сливаем найденные коробки с заказами
result = orders.join(selected_boxes, rsuffix = "_box")

In [31]:
result.to_excel(path.join(path.dirname(orders_filepath),'box_picked.xlsx'), encoding='cp1251')